## Sequential Multi-Agent Tracing Sample

Sample of tracking using OpenTelemetry with Azure Foundry Agent Services

### Setup Foundry Project Client

In [80]:
from typing import Any, Callable, Set, List
import re
import os
import time
import json
from azure.ai.projects import AIProjectClient
from azure.ai.agents.telemetry import trace_function
from azure.ai.agents.models import (
    FunctionTool,
    RequiredFunctionToolCall,
    SubmitToolOutputsAction,
    ToolOutput,
    ThreadMessage,
    MessageRole,
)
import azure.ai.agents as agentslib
import azure.ai.projects as projectslib
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor
from dotenv import load_dotenv, find_dotenv

# Your custom Python functions (for "fetch_datetime", etc.)
from utils.enterprise_functions import enterprise_fns

load_dotenv(dotenv_path=".env", override=True)

from utils.fdyauth import AuthHelper
settings = AuthHelper.load_settings()
credential = AuthHelper.test_credential()

if credential:
    print('Environment and authentication OK')
else:
    print("please login first")

Environment and authentication OK


In [81]:
# new AI Foundry Project resource endpoint / old azure ai services endpoint from the hub/project
project_client = AIProjectClient(
    credential=credential,
    endpoint=settings.project_endpoint,
    # api_version=os.environ["PROJECT_API_VERSION"]
)
print("project_client api version:", project_client._config.api_version)
print(f"azure-ai-agents version: {agentslib.__version__}")
print(f"azure-ai-projects version: {projectslib.__version__}")

project_client api version: 2025-05-15-preview
azure-ai-agents version: 1.1.0b3
azure-ai-projects version: 1.0.0b12


### Enable Azure Monitor Tracking

* get application insights connection string from the foundry project

In [82]:
# Enable Azure Monitor tracing geting the app insight connection string
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your AI Foundry project page.")
    exit()
configure_azure_monitor(connection_string=application_insights_connection_string)

In [83]:
scenario = "tracking_multiagent_scenario"
# scenario = os.path.basename(__file__)
tracer = trace.get_tracer(__name__)

print("using scenario:", scenario)
print("using tracer:", tracer)

using scenario: tracking_multiagent_scenario
using tracer: <opentelemetry.sdk.trace.Tracer object at 0x0000026F653E51C0>


### Set up the trace_function

* adding custom function trace to the OpenTelemetry by using decorator

In [84]:
@trace_function()
def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}

    # Adding attributes to the current span
    span = trace.get_current_span()
    span.set_attribute("requested_location", location)

    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json

@trace_function()
def convert_temperature(temperature: str) -> str:
    """
    Converts temperature between Celsius and Fahrenheit.
    
    :param temperature: Temperature string in format "25°C" or "77°F"
    :return: Converted temperature as JSON string
    :rtype: str
    """
    span = trace.get_current_span()
    span.set_attribute("input_temperature", temperature)
    
    try:
        value = float(''.join(filter(str.isdigit, temperature)))
        unit = 'C' if '°C' in temperature else 'F'
        
        if unit == 'C':
            converted = (value * 9/5) + 32
            result = f"{converted:.1f}°F"
        else:
            converted = (value - 32) * 5/9
            result = f"{converted:.1f}°C"
            
        return json.dumps({"converted_temperature": result})
    except Exception as e:
        return json.dumps({"error": f"Failed to convert temperature: {str(e)}"})

In [85]:
def check_for_celsius_in_messages(messages: List[ThreadMessage]) -> bool:
    """Helper function to check if any message contains a Celsius temperature"""
    for msg in messages:
        # print(f"msg type is {type(msg)}")
        try:
            # msg.context is a list , only checking the return message from the agent
            if hasattr(msg, 'role') and isinstance(msg.role, str) and msg.role == MessageRole.AGENT \
            and hasattr(msg, 'content') and '°C' in msg.content[0].text.value:    
                # re.compile(r'\b\d+°C\b'): # '°C' in msg.content:
                print(msg.content[0].text.value)
                return True
        except AttributeError:
            continue
    return False

In [86]:
def print_messages(messages: List[ThreadMessage]) -> None:
    """Helper function to print messages"""
    for message in messages:
        print(f"Role: {message.role}")
        print(f"Content: {message.content[0].text.value}")
        print("-" * 40)

In [87]:
# messages = project_client.agents.messages.list(thread_id="thread_Qp8oJRJEHSBvhraZ7IHzFSC7")
# result = check_for_celsius_in_messages(messages)
# print("Celsius found in messages:", result)

In [88]:
def process_agent_run(thread_id: str, run_id: str, functions: FunctionTool) -> None:
    """Helper function to process an agent run and handle tool calls"""
    run = project_client.agents.runs.get(thread_id=thread_id, run_id=run_id)
    
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.runs.get(thread_id=thread_id, run_id=run_id)
        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
            tool_calls = run.required_action.submit_tool_outputs.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project_client.agents.runs.cancel(thread_id=thread_id, run_id=run_id)
                break

            tool_outputs = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredFunctionToolCall):
                    try:
                        output = functions.execute(tool_call)
                        tool_outputs.append(
                            ToolOutput(tool_call_id=tool_call.id, output=output)
                        )
                    except Exception as e:
                        print(f"Error executing tool_call {tool_call.id}: {e}")

            if tool_outputs:
                project_client.agents.runs.submit_tool_outputs(
                    thread_id=thread_id,
                    run_id=run_id,
                    tool_outputs=tool_outputs
                )

        print(f"Current run status: {run.status}")
    
    return run

### Setup agent functions

In [89]:
# Initialize functions
user_functions: Set[Callable[..., Any]] = {fetch_weather, convert_temperature}
functions = FunctionTool(functions=user_functions)

### Sequential Agent Runs inside Tracing Scope

* Create two agents: weather assistant agent, and temperature conversion agent (Celsius and Fahrenheit)
* first run the weather assistant agent
* check output contains celsius, then run the temperature conversion agent subsequently
* use the same thread for both agent, so that the agent see the whole history (not recommended)
* return the messages from both agents run

In [90]:
with tracer.start_as_current_span(scenario):
    with project_client:
        # Create two agents with different roles
        # Create the weather assistant agent
        weather_agent = project_client.agents.create_agent(
            model=settings.model_deployment_name,
            name="weather-assistant",
#             instructions="""You are a helpful weather assistant. Follow these steps:
# 1. When asked about weather, politely acknowledge the request
# 2. Use the fetch_weather function to get the weather information
# 3. Present the weather information in a friendly, conversational way
# 4. If the temperature is in Celsius, mention that you'll ask for a conversion to Fahrenheit
# Be descriptive and natural in your responses.""",
            instructions="""You are a helpful weather assistant. Follow these steps:
1. When asked about weather, politely acknowledge the request
2. Use the fetch_weather function to get the weather information
3. Present the weather information in a friendly, conversational way
4. If the temperature is in Celsius, report back only the Celsius temperature
Be descriptive and natural in your responses.""",
            tools=functions.definitions,
        )
        print(f"Created weather agent, ID: {weather_agent.id}")

        # Create the temperature conversion agent
        conversion_agent = project_client.agents.create_agent(
            model=settings.model_deployment_name,
            name="conversion-assistant",
            instructions="""You are a helpful temperature conversion assistant. Follow these steps:
1. When asked to convert a temperature, acknowledge the request
2. Extract the temperature value from the previous messages
3. Use the convert_temperature function to perform the conversion
4. Present both temperatures in a clear, friendly way
Be detailed and explain the conversion clearly.""",
            tools=functions.definitions,
        )
        print(f"Created conversion agent, ID: {conversion_agent.id}")

        # Try block content for conversation display
        try:
            # Create a thread for the conversation
            thread = project_client.agents.threads.create()
            print(f"Created thread, ID: {thread.id}")

            # User asks about weather with a clear request
            message = project_client.agents.messages.create(
                thread_id=thread.id,
                role="user",
                # content="Can you tell me what the weather is like in New York today? I'd like to know the temperature in both Celsius and Fahrenheit, please.",
                content="Can you tell me what the weather is like in New York today? I'd like to know the temperature in Celsius °C please.",
            )
            print(f"Created initial user message, ID: {message.id}")

            # Start with the weather agent for the initial response
            weather_run = project_client.agents.runs.create(
                thread_id=thread.id,
                agent_id=weather_agent.id
            )
            print(f"Started weather agent run, ID: {weather_run.id}")

            # Process the weather agent's run
            weather_run = process_agent_run(thread.id, weather_run.id, functions)
            print(f"Weather agent run completed with status: {weather_run.status}")

            # Check messages for Celsius temperatures
            messages = list(project_client.agents.messages.list(thread_id=thread.id))
            print(f"Total messages in thread: {len(messages)}")
            # print_messages(messages)

            if check_for_celsius_in_messages(messages):
                
                # Create a message for the conversion agent with context
                conversion_msg = project_client.agents.messages.create(
                    thread_id=thread.id,
                    role="user",
                    content="Could you help convert this Celsius temperature to Fahrenheit so we can see both? Please explain the conversion."
                )
                print(f"Created conversion request message, ID: {conversion_msg.id}")

                # Start the conversion agent's run
                conversion_run = project_client.agents.runs.create(
                    thread_id=thread.id,
                    agent_id=conversion_agent.id
                )
                print(f"Started conversion agent run, ID: {conversion_run.id}")

                # Process the conversion agent's run
                conversion_run = process_agent_run(thread.id, conversion_run.id, functions)
                print(f"Conversion agent run completed with status: {conversion_run.status}")

            # Display the full conversation
            print("\nFull conversation:")
            final_messages = list(project_client.agents.messages.list(thread_id=thread.id))
            # print_messages(final_messages)

            final_messages.reverse()  # Show messages in chronological order
            for msg in final_messages:
                try:
                    if hasattr(msg, 'content'):
                        role = msg.role if hasattr(msg, 'role') else 'system'
                        print(f"{role}: {msg.content}")
                except Exception as e:
                    print(f"Error printing message: {e}")

        finally:
            # Clean up resources
            print("\nCleaning up resources...")
            project_client.agents.delete_agent(weather_agent.id)
            project_client.agents.delete_agent(conversion_agent.id)
            print("Agents deleted successfully")

Created weather agent, ID: asst_64kZlC7c7dZqXeCYGxLsP60G
Created conversion agent, ID: asst_TeFNSHai3hQW8lNfmuROOvv6
Created thread, ID: thread_iuARHaC80tF6sViDaKb5i5a6
Created initial user message, ID: msg_Rt1B0XO4Jifb0oSU6sKN62Gn
Started weather agent run, ID: run_FdqR2djpKRe28rQoG2jXukoM
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.COMPLETED
Weather agent run completed with status: RunStatus.COMPLETED
Total messages in thread: 2
The weather in New York today is sunny with a temperature of 25°C. It's a lovely day to enjoy the outdoors! Is there anything else you'd like to know about the weather?
Created conversion request message, ID: msg_kO4PBGDcC2cqzdFFOWFdFVGt
Started conversion agent run, ID: run_B5NfzNwBksAFDRoF850okIq7
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.COMPLETED
Conversion agent run completed with status: RunStatus.COMPLETED

Full conversation:
MessageRole.USER: [{'type': 'text', 'text': {'value': "Can you